# Tutorial 1: The assimilation step
DART-WRF is a python package which automates many things like configuration, saving configuration and output, handling computing resources, etc.

The data for this experiment is accessible for students on the server srvx1.


## Configuring the hardware
In case you use a cluster which is not supported, copy an existing cluster configuration and modify it, e.g. `config/jet.py`.


## Configuring the experiment
Firstly, you need to configure the experiment.
Copy the existing template and modify it `cp config/exp_template.py config/exp1.py`.

Customize your settings:

- expname should be a unique identifier and will be used as folder name
- model_dx is the model resolution in meters
- n_ens is the ensemble size
- update_vars are the WRF variables which shall be updated by the assimilation

```python
exp = utils.Experiment()
exp.expname = "test_newcode"
exp.model_dx = 2000
exp.n_ens = 40
exp.update_vars = ['U', 'V', 'W', 'THM', 'PH', 'MU', 'QVAPOR', 'QCLOUD', 'QICE', 'PSFC']

```

### Generating observations
In case you want to generate new observations, like for an observing system simulations experiment (OSSE), set 
```python
exp.use_existing_obsseq = False
```
in this case, you need to set the path to WRF nature run files from where DART can generate observations:
```python
exp.nature_wrfout_pattern = '/usr/data/sim_archive/exp_v1_nature/*/1/wrfout_d01_%Y-%m-%d_%H:%M:%S'
```

### Using pre-existing observation files

You can use pre-existing observation files with
```python
exp.use_existing_obsseq = '/usr/data/sim_archive/exp_ABC/obs_seq_out/%Y-%m-%d_%H:%M_obs_seq.out'
```
where times are filled, depending on the assimilation time.

### Customizing the DART namelist
By default, the DART namelist of the build directory will be used (copied). 
If you want to modify any parameters, specify your changes in a python dictionary like below. For a description of the parameters, see `the official DART documentation <https://docs.dart.ucar.edu/>`_.
```python
exp.dart_nml = {'&assim_tools_nml':
                    dict(filter_kind='1',
                         sampling_error_correction='.true.',
                        ),
                '&filter_nml':
                    dict(ens_size=exp.n_ens,
                         num_output_state_members=exp.n_ens,
                         num_output_obs_members=exp.n_ens,
                         inf_flavor=['0', '4'],
                         output_members='.true.',
                         output_mean='.true.',
                         output_sd='.true.',
                         stages_to_write='output',
                        ),
                '&quality_control_nml':
                    dict(outlier_threshold='-1',
                        ),
                '&location_nml':
                    dict(horiz_dist_only='.false.',
                '&model_nml':
                    dict(wrf_state_variables =
                        [['U',     'QTY_U_WIND_COMPONENT',     'TYPE_U',    'UPDATE','999',],
                         ['V',     'QTY_V_WIND_COMPONENT',     'TYPE_V',    'UPDATE','999',],
                         ['W',     'QTY_VERTICAL_VELOCITY',    'TYPE_W',    'UPDATE','999',],
                         ['PH',    'QTY_GEOPOTENTIAL_HEIGHT',  'TYPE_GZ',   'UPDATE','999',],
                         ['THM',   'QTY_POTENTIAL_TEMPERATURE','TYPE_T',    'UPDATE','999',],
                         ['MU',    'QTY_PRESSURE',             'TYPE_MU',   'UPDATE','999',],
                         ['QVAPOR','QTY_VAPOR_MIXING_RATIO',   'TYPE_QV',   'UPDATE','999',],
                         ['QICE',  'QTY_ICE_MIXING_RATIO',     'TYPE_QI',   'UPDATE','999',],
                         ['QCLOUD','QTY_CLOUDWATER_MIXING_RATIO','TYPE_QC', 'UPDATE','999',],
                         ['CLDFRA','QTY_CLOUD_FRACTION',       'TYPE_CFRAC','UPDATE','999',],
                         ['PSFC',  'QTY_SURFACE_PRESSURE',     'TYPE_PSFC', 'UPDATE','999',],
                         ['T2',    'QTY_2M_TEMPERATURE',       'TYPE_T',    'UPDATE','999',],
                         ['TSK',   'QTY_SKIN_TEMPERATURE',     'TYPE_T',    'UPDATE','999',],
                         ['REFL_10CM','QTY_RADAR_REFLECTIVITY','TYPE_REFL', 'UPDATE','999',]]),
                }
```
Any parameters in this dictionary will be overwritten compared to the default namelist.



### Single observation experiment
If you want to assimilate one observation, use 
```python
t = dict(plotname='Temperature', plotunits='[K]',
         kind='RADIOSONDE_TEMPERATURE', 
         n_obs=1,                    # number of observations
         obs_locations=[(45., 0.)],  # location of observations
         error_generate=0.2,    # observation error used to generate observations
         error_assimilate=0.2,  # observation error used for assimilation
         heights=[1000,],       # for radiosondes, use range(1000, 17001, 2000)
         loc_horiz_km=50,       # horizontal localization half-width
         loc_vert_km=2.5        # vertical localization half-width
        )  

exp.observations = [t,]  # select observations for assimilation
```

### Assimilating multiple observations
To generate a grid of observations, use
```python
vis = dict(plotname='VIS 0.6µm', plotunits='[1]',
           kind='MSG_4_SEVIRI_BDRF', sat_channel=1, 
           n_obs=961, obs_locations='square_array_evenly_on_grid',
           error_generate=0.03, error_assimilate=0.03,
           loc_horiz_km=50)
exp.observations = [t, vis,]
```

Caution, n_obs should only be one of the following:

- 22500 for 2km observation density/resolution 
- 5776 for 4km; 
- 961 for 10km; 
- 256 for 20km; 
- 121 for 30km

For vertically resolved data, like radar, `n_obs` is the number of observations at each observation height level.




## Configuring the assimilation experiment
We start by importing some modules:
```python
import datetime as dt
from dartwrf.workflows import WorkFlows
```

To assimilate observations at dt.datetime `time` we set the directory paths and times of the prior ensemble forecasts:

```python
prior_path_exp = '/users/students/lehre/advDA_s2023/data/sample_ensemble/'
prior_init_time = dt.datetime(2008,7,30,12)
prior_valid_time = dt.datetime(2008,7,30,12,30)
assim_time = prior_valid_time
```

To set up the experiment, call
```python
w = WorkFlows(exp_config='exp1.py', server_config='srvx1.py')
```
It will also create the output folders and backup the configuration files and scripts.


Finally, we run the data assimilation by calling
```python
w.assimilate(assim_time, prior_init_time, prior_valid_time, prior_path_exp)
```

Congratulations! You're done!